In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import einops
from collections import defaultdict
from typing import Callable, Optional
import os
import importlib
import pickle
import pandas as pd

import circuits.utils as utils
import circuits.othello_utils as othello_utils
import neuron_simulation.simulation_config as sim_config
import neuron_simulation.simulate_activations_with_dts as sim_activations

In [ ]:
default_config = sim_config.selected_config
device = "cpu"

# If desired, you can run simulate_activations_with_dts.py from this cell
# Values from default config will also be used later on in the notebook to filter out saved pickle files

# Example of filtering out certain configurations

# default_config.n_batches = 2

# for combination in default_config.combinations:
#     combination.ablate_not_selected = [True]

# sim_activations.run_simulations(default_config)

In [ ]:
def load_ablation_pickle_files(
    directory: str,
    dataset_size: int,
    ablation_method: str,
    ablate_not_selected: bool,
    add_error: bool,
):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".pkl") and "ablation" in filename:
            with open(os.path.join(directory, filename), "rb") as f:
                single_data = pickle.load(f)

            hyperparams = single_data["hyperparameters"]
            if hyperparams["ablate_not_selected"] != ablate_not_selected:
                continue

            if hyperparams["input_location"] != "mlp_neuron":
                if hyperparams["add_error"] != add_error:
                    continue

            if hyperparams["ablation_method"] != ablation_method:
                continue

            if hyperparams["dataset_size"] != dataset_size:
                continue
            data.append(single_data)
    return data


def extract_kl_divergence(
    data: list[dict],
    custom_function_name: str,
):
    nested_kl_div = {}
    for run in data:
        hyperparams = run["hyperparameters"]
        input_location = hyperparams["input_location"]
        trainer_id = hyperparams["trainer_id"]

        if input_location not in nested_kl_div:
            nested_kl_div[input_location] = {}
        if trainer_id not in nested_kl_div[input_location]:
            nested_kl_div[input_location][trainer_id] = {}

        for layer_tuple, func_results in run["results"].items():
            if custom_function_name in func_results:
                kl_div = func_results[custom_function_name]
                if len(layer_tuple) == 1:
                    layer = layer_tuple[0]
                    if layer not in nested_kl_div[input_location][trainer_id]:
                        nested_kl_div[input_location][trainer_id][layer] = []
                    nested_kl_div[input_location][trainer_id][layer].append(kl_div)
                else:
                    raise ValueError("Layer tuple length is not 1")

    return nested_kl_div


def min_kl_div_per_layer(nested_kl_div):
    min_kl_div = {}
    for input_location, trainer_ids in nested_kl_div.items():
        min_kl_div[input_location] = {}
        all_layers = set()
        for layer_results in trainer_ids.values():
            all_layers.update(layer_results.keys())

        for layer in all_layers:
            min_kl = float("inf")
            for trainer_id, layer_results in trainer_ids.items():
                if layer in layer_results:
                    min_kl = min(min_kl, min(layer_results[layer]))
            min_kl_div[input_location][layer] = min_kl

    return min_kl_div


def plot_kl_divergence(nested_kl_div):
    for input_location, trainer_ids in nested_kl_div.items():
        plt.figure(figsize=(12, 6))
        for trainer_id, layer_results in trainer_ids.items():
            layers = sorted(layer_results.keys())
            means = [np.mean(layer_results[layer]) for layer in layers]
            stds = [np.std(layer_results[layer]) for layer in layers]

            plt.errorbar(
                layers, means, yerr=stds, fmt="o-", capsize=5, label=f"Trainer {trainer_id}"
            )

        plt.xlabel("Layer")
        plt.ylabel("KL Divergence")
        plt.title(f"KL Divergence per Layer ({input_location}, {ablation_method} ablation)")
        plt.legend()
        plt.show()


def plot_min_kl_divergence(min_kl_div):
    plt.figure(figsize=(12, 6))
    for input_location, layer_results in min_kl_div.items():
        layers = sorted(layer_results.keys())
        kl_values = [layer_results[layer] for layer in layers]

        plt.plot(layers, kl_values, "o-", label=input_location)

    plt.xlabel("Layer")
    plt.ylabel("min KL Divergence")
    plt.title(f"min KL Divergence per Layer ({ablation_method} ablation)")
    plt.legend()
    plt.show()


directory = "decision_trees"
custom_function_name = (
    othello_utils.games_batch_to_input_tokens_flipped_bs_valid_moves_classifier_input_BLC.__name__
)

default_config = sim_config.selected_config
dataset_size = default_config.batch_size * default_config.n_batches

ablation_method = "dt"
ablate_not_selected = True
add_error = False

data = load_ablation_pickle_files(directory, dataset_size, ablation_method, ablate_not_selected, add_error)

example_dict = data[0]

print(example_dict["hyperparameters"].keys())

kl_div_per_layer = extract_kl_divergence(data, custom_function_name)
plot_kl_divergence(kl_div_per_layer)

In [ ]:
min_kl_div = min_kl_div_per_layer(kl_div_per_layer)
plot_min_kl_divergence(min_kl_div)

In [ ]:
def graph_3var_results(kl_div_per_layer: dict, eval_df: pd.DataFrame, layer: int, input_location: str):
    # Filter the evaluation dataframe
    eval_df_filtered = eval_df[eval_df['layer_idx'] == layer]
    
    # Prepare data for plotting
    l0 = []
    frac_recovered = []
    kl_values = []
    
    for trainer_id, layer_results in kl_div_per_layer[input_location].items():
        if layer in layer_results:
            # Find matching row in eval_df_filtered
            eval_row = eval_df_filtered[eval_df_filtered['trainer_idx'] == int(trainer_id)]
            if not eval_row.empty:
                l0.append(eval_row['l0'].values[0])
                frac_recovered.append(eval_row['frac_recovered'].values[0])
                kl_values.append(np.mean(layer_results[layer]))
    
    # Create the 2D plot
    plt.figure(figsize=(12, 8))
    
    # Plot the points
    scatter = plt.scatter(l0, frac_recovered, c=kl_values, cmap='viridis', s=50)
    
    # Set labels and title
    plt.xlabel('L0', fontsize=12)
    plt.ylabel('Fraction Recovered', fontsize=12)
    plt.title(f'L0 vs Fraction Recovered vs Neuron Simulation KL Divergence\n'
              f'Layer {layer}, {input_location}', fontsize=14)
    
    # Add a color bar
    cbar = plt.colorbar(scatter)
    cbar.set_label(f'Layer {layer} KL Divergence', fontsize=12)
    
    # Add grid lines
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Improve layout
    plt.tight_layout()

    plt.savefig(f"l0_vs_frac_recovered_vs_kl_divergence_{input_location}_layer_{layer}.png")
    
    # Show the plot
    plt.show()

# Load evaluation data
sae_mlp_out_eval = pd.read_csv("sae_eval_csvs/sae_mlp_out_feature_evaluations.csv")
transcoder_eval = pd.read_csv("sae_eval_csvs/transcoder_evaluations.csv")

# Example usage
layer = 1 

# Plot for sae_mlp_out_feature
graph_3var_results(kl_div_per_layer, sae_mlp_out_eval, layer, 'sae_mlp_out_feature')

# Plot for transcoder
graph_3var_results(kl_div_per_layer, transcoder_eval, layer, 'transcoder')

In [ ]:
# Currently broken, just need to get the dictionary keys right
# This can be used to plot multiple thresholds for a single location / custom function combination

# colors = ['b', 'g', 'r']  # Colors for different thresholds
# markers = ['o', 's', '^']  # Markers for different thresholds

# for custom_function in custom_functions:
#     function_name = custom_function.__name__
    
#     plt.figure(figsize=(12, 7))
    
#     for i, threshold in enumerate(thresholds):
#         f1_counts = accuracy_by_layer[function_name][threshold]
        
#         plt.plot(intervention_layers, f1_counts, color=colors[i], marker=markers[i], 
#                  label=f'Threshold: {threshold}', linewidth=2, markersize=8)
        
#         # Optionally, add value labels on each point
#         for j, count in enumerate(f1_counts):
#             plt.annotate(str(count), (intervention_layers[j], count), 
#                          textcoords="offset points", xytext=(0,5), ha='center', 
#                          fontsize=8, color=colors[i])
    
#     plt.title(f'Number of Neurons with F1 > Threshold for \n{function_labels[custom_function]} by Layer \n{dataset_size} datapoints Input location: {input_location} depth: {max_depth}', fontsize=14)
#     plt.xlabel('Layer Number', fontsize=12)
#     plt.ylabel('F1 Count', fontsize=12)
#     plt.legend(loc='best', fontsize=10)
#     plt.grid(True, linestyle='--', alpha=0.7)
    
#     plt.tight_layout()
#     output_filename = f"images/{input_location}_{function_name}_inputs_{dataset_size}_depth_{max_depth}_f1_count_by_layer_all_thresholds.png"
#     plt.savefig(output_filename, dpi=300, bbox_inches='tight')
#     plt.show()

# print("All graphs have been created and saved.")

In [ ]:
# Currently broken, just need to get the dictionary keys right

# layer = 1
# neuron_idx = 421
# custom_function_name = custom_functions[0].__name__

# decision_tree_filename = f"decision_trees/decision_trees_{input_location}_{dataset_size}.pkl"

# with open(decision_tree_filename, "rb") as f:
#     decision_tree = pickle.load(f)

# layer_dt = decision_tree[layer][custom_function_name]['decision_tree']['model']
# layer_dt = decision_tree[layer][custom_function_name]['binary_decision_tree']['model']

# games_BLC = train_data[custom_function_name]

# feature_names = []

# X_binary_train, X_binary_test, y_binary_train, y_binary_test = prepare_data(
#     games_BLC, binary_acts[layer]
# )
# accuracy, precision, recall, f1 = calculate_binary_metrics(
#     decision_tree[layer][custom_function_name]["binary_decision_tree"]["model"], X_binary_test, y_binary_test
# )

# print(f"Neuron 421 F1: {f1[neuron_idx]}")

# for i in range(games_BLC.shape[2]):
#     if i < 64:
#         square = idx_to_square_notation(i)
#         feature_names.append(f"Input_{square}")
#     elif i < 128:
#         j = i - 64
#         square = idx_to_square_notation(j)
#         feature_names.append(f"Occupied_{square}")
#     else:
#         feature_names.append(f"Output_{i}")

# print_decision_tree_rules(layer_dt, feature_names=feature_names, neuron_index=neuron_idx, max_depth=5)

